![NVIDIA Logo](images/nvidia.png)

# LLM Functions

In this notebook we introduce the idea of LLM functions where we wrap task-specific LLM behavior into functions to promote modularity and code reuse.

---

## Learning Objectives

By the time you complete this notebook you will be able to:
- Wrap task-specific LLM functionality in modular, reusable functions.

---

## Imports

In [ ]:
import json
import textwrap

from llm_utils.models import Models, LoraModels
from llm_utils.nemo_service_models import NemoServiceBaseModel

---

## List Models

In [ ]:
Models.list_models()

In [ ]:
LoraModels.list_models()

---

## Coding Best Practices in the Context of LLMs

Whether we are working with LLMs or not, we should adhere to good general coding practices.

Currently in this workshop our use of LLMs has primarily been for one-off tasks, but as we proceed through the workshop we will be wanting to compose the functionality of several LLMs that each perform specific tasks. To this end we should consider how we can organize the model functionality that we are and will be creating to facilitate our end goals.

Our approach will be to create what we will call **LLM functions**. Each LLM function will ecapsulate everything necessary to perform a specific **task** using an LLM.

### Many Good Approaches

We want to make clear that our approach here is by no means the only way to keep your LLM task functionality well organized. In your own codebases you will likely arrive at your own best practices. You may very well also be using 3rd party libraries such as [LangChain](https://www.langchain.com/) to help organize your LLM functionality.

In any case, you are advised to think through how to make your organization's LLM functionality well organized and well suited for optimal use and/or reuse.

In today's workshop we will adhere to a specific definition of an LLM function. In doing so we will better understand the role of model customization in the service of actual application functionality, and, will have a structure by which we can, as the workshop proceeds, accomplish many tasks both through the use of individual and composed LLM functions.

---

## Components of an LLM Function

In our approach, an LLM function will comprise of 2 main components:
1. An LLM instance
2. A prompt template

---

## LLM Instance

LLMs are the workhorses of our code bases today, so it goes without saying that an LLM function should leverage an LLM.

In the following examples we will use NeMo GPT43B.

In [ ]:
llm = NemoServiceBaseModel(Models.gpt43b.value)

---

## Prompt Templates

The specifc task that an LLM is intended to perform is well articulated by a prompt template. For example, given the same LLM, we might wish to perform summarization, question answering, translation, or classification. For each of these examples we could denote the task with an appropriate task template. Here we provide examples for each, using functions that expect the necessary arguments to contruct an appropriate prompt utilizing the prompt template.

### Summarization

In [ ]:
def summarization_template(text):
    return f'Summarize the following article:\n{text}'

### Question Answering

In [ ]:
def qa_template(context, question):
    return f'Given the context: {context}\nAnswer the question: {question}'

### Translation

In [ ]:
def translation_template(text, source_language, target_language):
    return f'Translate the following text from {source_language} to {target_language}:\n{text}'

### Classification

In [ ]:
def classification_template(text):
    return f'Classify the following text into "Spam" or "Not Spam":\n{text}'

---

## Making LLM Functions

Now that we have a model instance and several prompt templates, let's create a few naive LLM functions. Each function will leverage our model instance and one of the example prompt templates.

### Summarization

In [ ]:
def summarize(text):
    prompt = summarization_template(text)
    return llm.generate(prompt)

### Question Answering

In [ ]:
def qa(context, question):
    prompt = qa_template(context, question)
    return llm.generate(prompt)

### Translation

In [ ]:
def translate(text, source_language, target_language):
    prompt = translation_template(text, source_language, target_language)
    return llm.generate(prompt)

---

## Try Naive LLM Functions

Let's try out our LLM functions on the following paragraph espousing the use of modular code.

In [ ]:
modular_code = f"""\
Single-purpose modular functions represent a cornerstone of efficient and maintainable software development, \
embodying the principle of doing one thing and doing it well. By encapsulating a specific \
task or calculation within a self-contained unit of code, these functions enhance readability, \
facilitate debugging, and promote reuse across different parts of a project or even among different projects. \
This modular approach allows developers to build complex systems through the composition of simpler, \
well-understood pieces, significantly reducing the cognitive load required to understand or modify the system. \
Moreover, single-purpose functions make unit testing more straightforward, enabling developers to \
verify the correctness of each part in isolation before integrating them into a larger system. \
Adhering to this best practice not only speeds up the development process by enabling code reuse and parallel \
development but also contributes to the creation of more reliable and easily adaptable software systems.\
"""

### Summarization

In [ ]:
summarize(modular_code)

### Question Answering

In [ ]:
qa(modular_code, 'What is modular code\'s effect on debugging?')

### Translation

In [ ]:
translate(modular_code, 'English', 'Mandarin')

---

## LLM Function Constructor

The above LLM functions worked great, but to make their creation even easier going forward, we will provide the following `make_llm_function`.

In [ ]:
def make_llm_function(model, prompt_template_function, postprocessor=None):
    
    def llm_function(*prompt_template_args, **kwargs):

        prompt = prompt_template_function(*prompt_template_args)
        response = model.generate(prompt, **kwargs)

        if postprocessor:
            response = postprocessor(response)

        return response

    return llm_function

`make_llm_function` is a higher order function that will return an LLM function. As an improvement over our naive LLM functions above, LLM functions returned by `make_llm_function` can accept key word arguments to control LLM generation (`tokens_to_generate`, `top_k`, etc.), and also allows us to supply an optional `postprocessor` in cases where we want to postprocess model responses before returning them.

---

## Exercise: Make Translation LLM Function

For this exercise you are going to create a `translate` LLM function using the `make_llm_function` helper.

You will want to use the following, as arguments to `make_llm_function`:
1. An instance of GPT43B, provided just below as `llm`.
2. Our prompt template for the translation task, provided below as `translation_template`.
3. The postprocessing function `strip`, provided below will strip white space off the model's repsonse.

Feel free to check out the *Solution* below if you get stuck.

In [ ]:
llm = NemoServiceBaseModel(Models.gpt43b.value)

In [ ]:
def translation_template(text, source_language, target_language):
    return f'Translate the following text from {source_language} to {target_language}:\n{text}'

In [ ]:
def strip(text):
    return text.strip()

### Your Work Here

In [ ]:
translate = 'TODO' # Make a `translate` LLM function 

The following should work upon the successful implementation of `translate`.

In [ ]:
translate('I am learning a lot.', 'English', 'Spanish')

### Solution

In [ ]:
translate = make_llm_function(llm, translation_template, postprocessor=strip)

In [ ]:
translate('I am learning a lot.', 'English', 'Spanish')

---

## Exercise: Make Fixed Language Translation Function

The `translate` function you just created is very flexible and can translate to and from many languages. However, it's easy to imagine wanting a more specialized function that always translates from a specific language to a specific language. For this exercise you will create a new LLM function `translate_english_to_spanish` that will expect only a single argument, some English text, and will translate it to Spanish.

In order to do this you will need to create a new prompt template, based on a modification of `postprocess_translation` above that only expects a single `text` argument.

Feel free to check out the *Solution* below if you get stuck.

### Your Work Here

In [ ]:
translate_english_to_spanish = 'TODO' # 

The following should work upon the successful implementation of `translate_english_to_spanish`.

In [ ]:
translate_english_to_spanish('I am learning even more.')

### Solution

In [ ]:
def english_to_spanish_translation_template(text):
    return f'Translate the following text from English to Spanish:\n{text}'

In [ ]:
translate_english_to_spanish = make_llm_function(llm, english_to_spanish_translation_template, postprocessor=strip)

In [ ]:
translate_english_to_spanish('I am learning even more.')